In [1]:
import numpy as np
from scipy import interpolate
from scipy import integrate

In [2]:
def overturning_scale_depth(rho_l, rho_h, z):
    """
    Calculates the overturning depth scale (H) based on Eq. (8) of De Boer et al., (2010)
    
    Reference:
    
    De Boer, A. M., Gnanadesikan, A., Edwards, N. R., & Watson, A. J. (2010). Meridional density gradients do 
    not control the Atlantic overturning circulation. Journal of Physical Oceanography, 40(2), 368-380.
    
    Input:
    
    rho_l = vertical profile of density in low latitudes  [ kg m^{-3} ]
    rho_h = vertical profile of density in high latitudes [ kg m^{-3} ]
    z     = depth [ m ]
    
    Output: 
    
    H = overturning depth scale [ m ]
    
    """
    
    # calculate \Delta_y \rho(z)
    delta_y_rho_z = (rho_h - rho_l)
    
    # remove nan values
    isn = np.isnan(delta_y_rho_z)
    delta_y_rho_z_isn = delta_y_rho_z[~isn]
    z_isn = z[~isn]
    
    # calculate vertical integral
    delta_y_rho_z_int = integrate.cumtrapz(delta_y_rho_z_isn, x=z_isn, initial=0.)
    delta_y_rho_z_int_avg = np.array( np.trapz(delta_y_rho_z_int,z_isn) / z_isn[-1] )
    
    # interpolate to a finer grid for more accuracy
    x = z_isn
    y = delta_y_rho_z_int
    f = interpolate.interp1d(x, y, kind='linear')
    z_r = np.linspace(np.min(z_isn),np.max(z_isn),10000)
    delta_y_rho_z_int_r = f(z_r)
    
    # find depth where delta_y_rho_z_int_avg == delta_y_rho_z_int_r
    H = z_r[ np.where(delta_y_rho_z_int_avg>=delta_y_rho_z_int_r)[0][-1] ]
    
    return H

In [3]:
# sample potential density data referenced to 2000 dbar from a historical ACCESS-CM2 simulation

rho_l = np.array([1032.08939212, 1032.1050324 , 1032.14778428, 1032.29054195,
       1032.55238182, 1032.88770717, 1033.18234337, 1033.43478368,
       1033.64524788, 1033.82663975, 1033.98529005, 1034.12237614,
       1034.23750932, 1034.33116131, 1034.40700347, 1034.47117316,
       1034.52699996, 1034.5764456 , 1034.62196156, 1034.66550623,
       1034.70940662, 1034.76372387, 1034.8419734 , 1034.94720326,
       1035.07848059, 1035.23198989, 1035.40021547, 1035.57785765,
       1035.7525586 , 1035.91658212, 1036.06683425, 1036.20105487,
       1036.3172561 , 1036.4155532 , 1036.49862838, 1036.5687504 ,
       1036.62959236, 1036.68346801, 1036.73040097, 1036.77062151,
       1036.80437546, 1036.83264199, 1036.85695024, 1036.87336617,
       1036.88502771, 1036.89130251, 1036.89676437, 1036.90343791,
       1036.90593775, 1036.91106344])

rho_h = np.array([1034.87749515, 1034.90194343, 1034.98449259, 1035.11794443,
       1035.2408348 , 1035.4305272 , 1035.52151136, 1035.58697371,
       1035.64361376, 1035.68420364, 1035.71335438, 1035.74163341,
       1035.76196332, 1035.78637369, 1035.79795328, 1035.81877778,
       1035.83606036, 1035.84926795, 1035.85498304, 1035.86018879,
       1035.8649258 , 1035.87141088, 1035.87845894, 1035.88926157,
       1035.90754276, 1035.93663622, 1035.98075618, 1036.03831198,
       1036.10110914, 1036.16160756, 1036.22251414, 1036.28486074,
       1036.35343534, 1036.41835066, 1036.4908909 , 1036.55732797,
       1036.61684793, 1036.66940116, 1036.71137527, 1036.74212886,
       1036.76083486, 1036.80036181, 1036.84296138, 1036.87620749,
       1036.8838289 , 1036.88523292, 1036.90283835, 1036.89512208,
                 np.nan,           np.nan])

z = np.array([5.00000000e+00, 1.50000000e+01, 2.50000000e+01, 3.50000000e+01,
       4.50000000e+01, 5.50000000e+01, 6.50000000e+01, 7.50000000e+01,
       8.50000000e+01, 9.50000000e+01, 1.05000000e+02, 1.15000000e+02,
       1.25000000e+02, 1.35000000e+02, 1.45000000e+02, 1.55000000e+02,
       1.65000000e+02, 1.75000000e+02, 1.85000000e+02, 1.95000000e+02,
       2.05000000e+02, 2.16846756e+02, 2.41349014e+02, 2.80780731e+02,
       3.43250458e+02, 4.27315552e+02, 5.36715637e+02, 6.65414124e+02,
       8.12781616e+02, 9.69065125e+02, 1.13093494e+03, 1.28960461e+03,
       1.45577014e+03, 1.62292566e+03, 1.80155811e+03, 1.98485461e+03,
       2.18290479e+03, 2.38841748e+03, 2.61093506e+03, 2.84256445e+03,
       3.09220483e+03, 3.35129468e+03, 3.62805762e+03, 3.91326440e+03,
       4.21449512e+03, 4.52191797e+03, 4.84256592e+03, 5.16612988e+03,
       5.49924512e+03, 5.83129443e+03])

In [4]:
H = overturning_scale_depth(rho_l, rho_h, z)

print('H = ' + str(np.round(H)) + ' meters')

H = 908.0 meters
